In [2]:
from catboost import CatBoostClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
import preprocessing as pp
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os

In [7]:

train_all_data=[]
test_all_data=[]
train_all_labels=[]
test_all_labels=[]


for i in range(7):
    for j in range(1, 5):
        if os.path.exists(f'train_stiffness/obj{i}_{j}.txt.npy'):
            data = np.load(f'train_stiffness/obj{i}_{j}.txt.npy')
            print(data.shape)
            point_split = int(0.9 * len(data))
            train_all_data.append(data[:point_split,:,:])
            test_all_data.append(data[point_split:,:,:])
            train_all_labels.append(np.array([i]*len(data[:point_split,:,:])))
            test_all_labels.append(np.array([i]*len(data[point_split:,:,:])))

(389, 30, 96)
(308, 30, 96)
(422, 30, 96)
(279, 30, 96)
(739, 30, 96)
(858, 30, 96)
(955, 30, 96)
(794, 30, 96)
(679, 30, 96)
(512, 30, 96)
(163, 30, 96)
(81, 30, 96)


In [9]:
print(np.unique(train_all_labels, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([1257,  665,  772,  859,  714,  611,  678]))


In [8]:
train_all_data = np.concatenate(train_all_data)
test_all_data = np.concatenate(test_all_data)
print(train_all_data.shape)
print(test_all_data.shape)

train_all_labels = np.concatenate(train_all_labels)
test_all_labels = np.concatenate(test_all_labels)
print(train_all_labels.shape)
print(test_all_labels.shape)

(5556, 30, 96)
(623, 30, 96)
(5556,)
(623,)


In [10]:
temp_shape_train = train_all_data.shape
train_all_data_resaped = train_all_data.reshape(temp_shape_train[0]*temp_shape_train[1], temp_shape_train[2])
temp_shape_test = test_all_data.shape
test_all_data_resaped = test_all_data.reshape(temp_shape_test[0]*temp_shape_test[1], temp_shape_test[2])

In [11]:
ss=StandardScaler()
ss.fit(train_all_data_resaped)

,copy,True
,with_mean,True
,with_std,True


In [14]:
sums = np.sum(train_all_data_resaped, axis=0)
positions = np.argsort(sums)[::-1]

In [15]:
positions

array([15, 93, 63, 31, 47, 71, 76, 39, 69, 57, 44, 24, 35, 82, 68, 87, 77,
       92, 55, 12, 28, 60, 89, 83, 75, 56, 78, 54, 91, 86, 88, 52, 90, 85,
       79, 70, 37, 53, 17, 72, 80, 65, 66, 14, 84, 64, 38, 73, 48, 29, 50,
       61, 67, 58, 32, 27, 74, 30, 34, 42, 59, 43, 13, 81, 45, 95, 41, 46,
       62, 49, 94, 51, 33, 36, 40, 22, 25, 23, 21, 26, 16, 18, 19, 20, 11,
       10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [12]:
ss_train_all_data_resaped=ss.transform(train_all_data_resaped)
ss_test_all_data_resaped=ss.transform(test_all_data_resaped)

In [130]:
sums[35]

np.float64(-1.833681428919931e-09)

In [13]:
ss_train_all_data_resaped_returned = ss_train_all_data_resaped.reshape(-1, 30, 96)
ss_test_all_data_resaped_returned = ss_test_all_data_resaped.reshape(-1, 30, 96)

In [14]:
def extract_features(X):
    """X shape: (20, 96) -> returns feature vector"""
    features = []

    # 1. Статистические
    stats = np.vstack([
        X.mean(axis=0), 
        #X.std(axis=0), 
        #np.median(X, axis=0),
        X.max(axis=0), 
        #X.min(axis=0), 
        #np.ptp(X, axis=0),
        #iqr(X, axis=0), 
        #median_abs_deviation(X, axis=0)
    ]).flatten()    
    # 2. Динамические
    diffs = np.diff(X, axis=0)
    dynamics = np.vstack([
         diffs.mean(axis=0), 
    #    diffs.std(axis=0), 
    #    np.max(np.abs(diffs), axis=0)
    ]).flatten()
    
    # 3. Частотные (первые 3 гармоники)
    #fft = np.abs(np.fft.fft(X, axis=0))[1:4].flatten()
    
    return np.concatenate([stats])

In [15]:
included_indices = [i for i in range(12,96) if i != 15]

In [16]:
X_train_extracted = np.array([extract_features(x) for x in ss_train_all_data_resaped_returned[:, :, included_indices]])
X_test_extracted = np.array([extract_features(x) for x in ss_test_all_data_resaped_returned[:, :, included_indices]])
print(X_train_extracted.shape)

(5556, 166)


In [189]:
weights_dict = {0: 1.2, 1:1, 2:1.1, 3:1, 4:1}

In [18]:
model = CatBoostClassifier(
    iterations=1500,
    classes_count=7,
    #class_weights=weights_dict,
    learning_rate=0.1,
    early_stopping_rounds = 200,
    depth=4,
    eval_metric='Accuracy',
    loss_function='MultiClass',
    l2_leaf_reg=10,
    #bootstrap_type = 'Poisson',
    #subsample = 0.8,
    verbose=100,
    task_type='CPU',
)

model.fit(X_train_extracted.squeeze(), train_all_labels, eval_set=(X_test_extracted.squeeze(), test_all_labels))

0:	learn: 0.5410367	test: 0.4414125	best: 0.4414125 (0)	total: 220ms	remaining: 5m 29s
100:	learn: 0.8846292	test: 0.6147673	best: 0.6147673 (100)	total: 5.18s	remaining: 1m 11s
200:	learn: 0.9267459	test: 0.6581059	best: 0.6597111 (158)	total: 10.1s	remaining: 1m 5s
300:	learn: 0.9481641	test: 0.6709470	best: 0.6757624 (257)	total: 15s	remaining: 59.6s
400:	learn: 0.9573434	test: 0.6982343	best: 0.6982343 (385)	total: 19.9s	remaining: 54.5s
500:	learn: 0.9604032	test: 0.7062600	best: 0.7062600 (469)	total: 24.7s	remaining: 49.3s
600:	learn: 0.9659827	test: 0.7110754	best: 0.7126806 (559)	total: 29.8s	remaining: 44.6s
700:	learn: 0.9710223	test: 0.7191011	best: 0.7191011 (690)	total: 34.7s	remaining: 39.5s
800:	learn: 0.9737221	test: 0.7239165	best: 0.7239165 (750)	total: 40s	remaining: 34.9s
900:	learn: 0.9780418	test: 0.7287319	best: 0.7303371 (860)	total: 45s	remaining: 29.9s
1000:	learn: 0.9798416	test: 0.7255217	best: 0.7303371 (860)	total: 50.7s	remaining: 25.3s
Stopped by overfi

In [19]:
X_for_pred = X_test_extracted 
y_true = test_all_labels


y_preds = model.predict(X_for_pred.squeeze())
print("Classification Report:")
print(classification_report(y_true, y_preds))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_preds))

print(f"Accuracy: {accuracy_score(y_true, y_preds):.4f}")

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       141
           1       0.98      0.62      0.76        74
           2       0.57      0.40      0.47        86
           3       0.42      0.72      0.53        96
           4       0.77      0.61      0.68        80
           5       0.76      0.76      0.76        68
           6       0.80      0.82      0.81        78

    accuracy                           0.73       623
   macro avg       0.76      0.70      0.72       623
weighted avg       0.77      0.73      0.74       623

Confusion Matrix:
[[141   0   0   0   0   0   0]
 [  0  46   8  19   0   1   0]
 [  0   1  34  51   0   0   0]
 [  0   0  12  69  14   1   0]
 [  0   0   6  24  49   1   0]
 [  0   0   0   0   0  52  16]
 [  0   0   0   0   1  13  64]]
Accuracy: 0.7303


In [22]:
model.save_model('train_stiffness/catboost_stiffness.cbm') 

In [23]:
import pickle
with open('train_stiffness/scaler_stiffness.pkl','wb') as f:
    pickle.dump(ss, f)

In [ ]:
classes = {0:'Бутылка', 1:'Яблоко', 2:'Губка', 3:'Протез открыт', 4:'Протез закрыт'}


Бутылка
